In [2]:
import socket

game_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
game_socket.connect(('127.0.0.1', 12345))

rx_msg_str = ""
while "\n" not in rx_msg_str:
    rx_msg_str += game_socket.recv(1024).decode()

current_word_guessed = ""
if rx_msg_str[:5] == "START":
    header, length_word = rx_msg_str[:-1].split(",")
    current_word_guessed = "?"*int(length_word)
    print("Current word guessed: " + current_word_guessed)
    
game_over = False
while not game_over:
    print("Menu:")
    print("1) Guess a letter")
    print("2) Guess a word")
    print("3) Quit")
    
    opt = input("Type an option:")
    while len(opt)!=1 and opt not in "123":
        opt = input("Type a valid option again:")
    
    if opt == "1":
        answer = ""
        while len(answer)!=1 and not answer.isalpha():
            answer = input("Type a letter: ").lower()
        tx_msg = "GUESS,"+answer+"\n"
    elif opt == "2":
        answer = ""
        while len(answer)==0 and not answer.isalpha():
            answer = input("Type a word: ").lower()        
        tx_msg = "HWORD,"+answer+"\n"
    else:
        tx_msg = "QUIT\n"
    game_socket.send(tx_msg.encode())
        
    # wait for server feedback
    rx_msg_str = ""
    while "\n" not in rx_msg_str:
        rx_msg_str += game_socket.recv(1024).decode()
    
    if rx_msg_str[:5] == "GUESS":
        guess_msgs = []
        if ',' in rx_msg_str:
            guess_msgs = rx_msg_str[6:-1].split(",")
        for i in guess_msgs:
            current_word_guessed = current_word_guessed[:int(i)] + answer + current_word_guessed[int(i)+1:]
        print("Current word guessed: " + current_word_guessed)
    if rx_msg_str[:-1] == "WIN":
        print("Current word guessed: " + current_word_guessed)
        game_over = True
        print("You win!")
    if rx_msg_str[:-1] == "LOSE":
        print("Current word guessed: " + current_word_guessed)
        game_over = True
        print("You lose!")
    if rx_msg_str[:-1] == "QUIT":
        game_over = True
        print("You quit!")

game_socket.close()

Current word guessed: ??????
Menu:
1) Guess a letter
2) Guess a word
3) Quit
Type an option:1
Type a letter: k
Current word guessed: k?????
Menu:
1) Guess a letter
2) Guess a word
3) Quit
Type an option:3
You quit!
